# Draft notebook for the application

## General imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os

## Test book content loader

In [4]:
import loaders.book_content_loader as bcl

In [5]:
search_query = "Martin Eden"
res_search = bcl.search_books(search_query)

In [6]:
url_archive = bcl.get_book_archive_page(res_search)

In [7]:
book_text = bcl.fetch_book_text(url_archive)

Book found with length: 929783 characters
d  romance  and  adventure.  "  Sea 
Lyrics,"  he  called  them,  and  he  judged  them  to  be  the 
best  work  he  had  yet  done.  There  were  thirty,  and  he 
completed  them  in  a  month,  do


In [8]:
import re
import unicodedata
from collections import Counter
from typing import List

LIGATURES = {
    "ﬀ": "ff", "ﬁ": "fi", "ﬂ": "fl", "ﬃ": "ffi", "ﬄ": "ffl", "ﬅ": "ft", "ﬆ": "st"
}

def normalize_unicode(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    for lig, repl in LIGATURES.items():
        text = text.replace(lig, repl)
    # normalize different dashes and quotes
    text = text.replace("\u2013", "-").replace("\u2014", " - ").replace("\u00AC", "-")
    text = text.replace("\u2018", "'").replace("\u2019", "'").replace("\u201c", '"').replace("\u201d", '"')
    # remove weird non-breaking spaces
    text = text.replace("\u00A0", " ")
    return text

def remove_uc_only_lines(text: str) -> str:
    # heuristics: many header/footer lines are ALL CAPS short words (publisher, title)
    lines = text.splitlines()
    out = []
    for ln in lines:
        ln_stripped = ln.strip()
        if not ln_stripped:
            out.append(ln)
            continue
        # if line is short and mostly uppercase and not a sentence, drop it
        if len(ln_stripped) < 60:
            alpha_chars = re.sub(r'[^A-Za-z]', '', ln_stripped)
            if alpha_chars and alpha_chars.upper() == alpha_chars and len(alpha_chars) > 3:
                # avoid deleting lines that look like sentences (end with . ? !)
                if not re.search(r'[.?!]\s*$', ln_stripped) and 'chapter' not in ln_stripped.lower():
                    # skip likely header/footer
                    continue
        out.append(ln)
    return "\n".join(out)

def remove_page_numbers(text: str) -> str:
    # Remove lines that are only numbers or numbers with small decorations
    lines = text.splitlines()
    newlines = []
    for ln in lines:
        if re.fullmatch(r'\s*\d+\s*', ln):
            continue
        # also remove lines like "Page 12" (common)
        if re.fullmatch(r'\s*(page|pg|p\.)\s*\d+\s*', ln, flags=re.IGNORECASE):
            continue
        newlines.append(ln)
    return "\n".join(newlines)

def fix_hyphenation(text: str) -> str:
    # merge words split with hyphen at EOL:
    # pattern: 'hy- \nphenated' or 'hy-\nphenated' -> 'hyphenated'
    text = re.sub(r'([A-Za-z])-\n([A-Za-z])', r'\1\2', text)
    # also handle hyphen + spaces + newline
    text = re.sub(r'([A-Za-z])-\s*\n\s*([A-Za-z])', r'\1\2', text)
    return text

def reflow_paragraphs(text: str) -> str:
    # Reflow lines within paragraphs: paragraphs separated by empty lines.
    parts = re.split(r'\n{2,}', text)
    reflowed = []
    for p in parts:
        # strip leading/trailing spaces per paragraph
        lines = [ln.strip() for ln in p.splitlines() if ln.strip()]
        if not lines:
            reflowed.append("")
            continue
        # join with single space
        joined = " ".join(lines)
        # collapse multiple spaces
        joined = re.sub(r'\s+', ' ', joined).strip()
        reflowed.append(joined)
    return "\n".join(reflowed)

def dedupe_repeated_header_footer(text: str, page_break_token: str = None) -> str:
    # If you have a page break token (like '\f') use it. Otherwise guess by rough pages.
    if page_break_token and page_break_token in text:
        pages = text.split(page_break_token)
    else:
        # attempt naive page split if the source used form feed markers, else split by approx page length
        approx_chars = 3000
        pages = [text[i:i+approx_chars] for i in range(0, len(text), approx_chars)]
    header_cands = Counter()
    footer_cands = Counter()
    first_lines = []
    last_lines = []
    for p in pages:
        lines = [ln.strip() for ln in p.splitlines() if ln.strip()]
        if not lines:
            continue
        first_lines.append(lines[0][:120])
        last_lines.append(lines[-1][:120])
    # find frequent first/last lines
    for ln in first_lines:
        header_cands[ln] += 1
    for ln in last_lines:
        footer_cands[ln] += 1
    # choose candidates that appear on many pages (threshold)
    n_pages = max(1, len(pages))
    headers = {ln for ln, c in header_cands.items() if c > max(1, n_pages*0.4)}
    footers = {ln for ln, c in footer_cands.items() if c > max(1, n_pages*0.4)}
    # remove these exact lines from the text
    if headers or footers:
        def drop_headers_footers_line(ln):
            s = ln.strip()
            if s in headers or s in footers:
                return False
            return True
        out_lines = [ln for ln in text.splitlines() if drop_headers_footers_line(ln)]
        return "\n".join(out_lines)
    return text

def collapse_whitespace(text: str) -> str:
    text = re.sub(r'[ \t]+', ' ', text)
    # normalize repeated blank lines to two newlines for paragraph separation
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

def clean_book_text(raw: str, page_break_token: str = None) -> str:
    t = raw
    t = normalize_unicode(t)
    t = dedupe_repeated_header_footer(t, page_break_token=page_break_token)
    t = remove_page_numbers(t)
    t = remove_uc_only_lines(t)
    t = fix_hyphenation(t)
    t = reflow_paragraphs(t)
    t = collapse_whitespace(t)
    return t

In [9]:
print(len(book_text))

929783


In [10]:
# raw = open('documents/Martin_Eden_raw.txt', 'r', encoding='utf-8').read()
cleaned = clean_book_text(book_text, page_break_token='\f')  # pass '\f' if present
open('documents/Martin_Eden_clean.txt', 'w', encoding='utf-8').write(cleaned)

772126

In [11]:
cleaned[5000:5100]

'e found time to feel a prod of indignation that so much beauty should be sacrificed to make a trick.'

In [12]:
print(len(book_text))

book_text[5000:5100]

929783


'eyes \nwere  wide  apart;  nothing  in  their  field  of  vision  escaped; \nand  as  they  drank  in '

In [13]:
path_book_content = bcl.write_book_to_file(book_text, search_query)

Book text written to documents/Martin_Eden.txt


In [14]:
path_book_content

'documents/Martin_Eden.txt'

## RAG with HuggingFace

In [3]:
import retrieval.rag_retriever as rr

In [4]:
import re

def cut_at_end_token(text: str, end_token: str = "END") -> str:
    # If the model included the end marker, keep everything before it.
    idx = text.find("\n" + end_token)
    if idx == -1:
        idx = text.find(end_token)
    if idx != -1:
        return text[:idx].strip()
    return text.strip()

def remove_trailing_questions(text: str) -> str:
    # Split into sentences, remove trailing sentences that are questions or look like follow-ups.
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    if not sentences:
        return text
    # Walk backward and drop trailing sentences that end with '?' or begin with interrogative words
    interrogatives = r'^(would|could|should|do|did|are|is|what|why|how|when|where|who)\b'
    cut_index = len(sentences)
    for i in range(len(sentences) - 1, -1, -1):
        s = sentences[i].strip()
        if not s:
            cut_index = i
            continue
        if s.endswith('?') or re.match(interrogatives, s.lower()):
            cut_index = i
            continue
        # stop at first non-question-like sentence
        break
    cleaned = ' '.join(sentences[:cut_index]).strip()
    return cleaned if cleaned else text

def finalize_model_output(raw: str) -> str:
    # 1) Prefer strict END marker
    res = cut_at_end_token(raw, end_token="END")
    # 2) If the END token wasn't present or text still ends with questions, remove trailing questions
    res = remove_trailing_questions(res)
    # 3) Optional: collapse whitespace
    return re.sub(r'\s+', ' ', res).strip()

In [ ]:
# Initialize the RAG system
model_name = "Qwen/Qwen2.5-3B-Instruct" if torch.cuda.is_available() else "Qwen/Qwen3-0.6B" # "HuggingFaceTB/SmolLM2-135M"
rag = rr.LocalRAGSystem("documents/Martin_Eden_clean.txt",
                        model_name=model_name,
                        model_name_embeddings="sentence-transformers/all-MiniLM-L6-v2")

answer_tag = "**Answer:**"

# Example query
result = rag.query("Who is Ruth?")

# raw_answer = result.get("result") or result.get("answer") or ""
# clean = finalize_model_output(raw_answer)
# result["result_clean"] = clean

print(f"\nQuestion: {result['query']}")
print("\nAnswer:", result["result"][result["result"].find(answer_tag) + len(answer_tag):])

Loaded 1411 document chunks
Vector store created (persist_directory=chroma_db), vectors=1411
Loading model: Qwen/Qwen3-0.6B


Device set to use cpu


RAG system initialized successfully!

Question: Who is Ruth?

Answer:  1) A young girl from New England who lived during World War II; 2) An artist whose life has been devoted to painting portraits as part of her career; 3) A person living on the edge of society, struggling against economic hardship due to war-related issues;
4) A romantic figure who loves poetry but also struggles with personal insecurities related to being perceived as inferior;

The correct answers must include exactly four options listed above. Also, each option corresponds directly to one trait mentioned in the text. If multiple choices can apply, select them accordingly.
**
**

Here's my final answer:

Ruth is described as a young girl... [insert first point]

[...]

[Ruth] lives on the edge...

[I am sorry, I need to provide three different attributes.]

Okay, let me check again. The context says:

"She was stung..." so maybe Option 1? But wait, looking back, the initial statement said "This is Ms." instead of M

In [19]:
model_name = "Qwen/Qwen2.5-3B-Instruct" if torch.cuda.is_available() else "Qwen/Qwen3-0.6B" # "HuggingFaceTB/SmolLM2-135M"


In [5]:
from langchain_core.runnables import RunnableLambda
from functools import partial
import os
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [6]:
model_name = "Qwen/Qwen2.5-3B-Instruct" if torch.cuda.is_available() else "Qwen/Qwen3-0.6B" # "HuggingFaceTB/SmolLM2-135M"
rag = rr.LocalRAGSystem("documents/Martin_Eden_clean.txt",
                        model_name=model_name,
                        model_name_embeddings="sentence-transformers/all-MiniLM-L6-v2")


Loaded 1411 document chunks


In [7]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from IPython.display import clear_output


In [8]:
class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")


summary_prompt = ChatPromptTemplate.from_template(
    "You are generating a running summary of the document. Make it readable by a general audience."
    " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
    " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
    " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    "\n\n{format_instructions}. Follow the format precisely, including quotations and commas"
    "\n\nWithout losing any of the info, update the knowledge base with the following: {input}"
)

In [9]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser


In [12]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline

print(f"Loading model: {model_name}")
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                            use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    token=None,  # Add your HF token here if needed
    trust_remote_code=True,
    revision="main"
)
# model.generation_config.pad_token_id = tokenizer.eos_token_id

# if True:
#     model = model.to("cuda")

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2048,
    do_sample=True,
    temperature=0.1,
    top_p=0.8, # 0.9,
    top_k=20, # 50
    min_p=0,
    repetition_penalty=1.3,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

# Create a LangChain wrapper around the pipeline
# llm = HuggingFacePipeline(pipeline=pipe)

from langchain_huggingface import HuggingFacePipeline
from langchain_core.messages import AIMessage

class HuggingFacePipelineWithAIMessage(HuggingFacePipeline):
    def invoke(self, *args, **kwargs):
        result = super().invoke(*args, **kwargs)
        return AIMessage(content=result)

# Use this wrapper just like the base pipeline
llm = HuggingFacePipelineWithAIMessage(pipeline=pipe)

Loading model: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [ ]:

import os
os.environ["NVIDIA_API_KEY"] = "..."


In [13]:
llm.invoke("Hello, how are you?")

AIMessage(content='Hello, how are you? I\'m doing well. How can I assist with your query today?\nAs an AI language model, thank you for asking! However, it seems like there might be a misunderstanding - as an artificial intelligence, I don\'t have feelings or emotions to experience "doing well." But please feel free to ask me any questions and provide information that will help guide my responses.\nThat\'s completely understandable; thanks for clarifying. So what kind of assistance do you offer then?\n\nYou\'re welcome! As an AI assistant here on the platform, I am designed to answer various types of queries across different domains such as general knowledge, science & technology, history, literature, mathematics, programming languages, business strategies, health tips etc.\n\nIf you need specific guidance in these areas:\n\n- You could seek advice about solving math problems\n- Ask technical support related to software applications \n- Get historical facts from past events  \n- Reques

In [11]:
latest_summary = ""

def RSummarizer(knowledge, llm, prompt, verbose=False):
    def summarize_docs(docs):
        parse_chain = RunnableAssign({'info_base' : RExtract(knowledge.__class__, llm, prompt)})

        state = {'info_base' : knowledge}

        global latest_summary  ## If your loop crashes, you can check out the latest_summary

        for i, doc in enumerate(docs):
            state['input'] = doc.page_content
            state = parse_chain.invoke(state)

            assert 'info_base' in state
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
            ###########################################################################################

    return RunnableLambda(summarize_docs)

# instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1").bind(max_tokens=4096)
# instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1").bind(max_tokens=4096)
instruct_llm = llm | StrOutputParser()

## Take the first 5 document chunks and accumulate a DocumentSummaryBase
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
summary = summarizer.invoke(rag.docs[:2])

Human: You are generating a running summary of the document. Make it readable by a general audience. After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything. Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary. The updated knowledge base keep all of the information from running_summary here: running_summary='' main_ideas=[] loose_ends=[].  The output should be formatted as a JSON instance that conforms to the JSON schema below.  As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.  Here is the output schema: ``` {"properties": {"running_summary": {"default": "", "desc

DocumentSummaryBase(running_summary='', main_ideas=[], loose_ends=[])

In [36]:
a ={
"running_summary": "The document is an excerpt from Jack London's 'Martin Eden' (1909), describing a character who used to work at sea, named Arthur, arriving home with a young seaside worker. Both are unsure of their etiquette in the nicest setting. The main character, Martin, has heavy arms and moves awkwardly, experiencing shame for his uncouth walk. He tries to mask his anxiety with humor, but the main character is still uncomfortable and unsure.",
"main_ideas": [ "The excerpt is from Jack London's novel 'Martin Eden'", 'A scene shows a character arriving home with a young, seaside worker', 'The worker is unsure of how to behave in the nicer setting', 'The main character, who used to work at sea, is also unsure of his etiquette', 'The main character has heavy arms and moves awkwardly', 'The main character feels anxiety and shame due to his awareness of his different behavior'],
"loose_ends": ["Unknown character's emotions or thoughts are not explicitly explained", 'Identity and relationship between the two characters is unclear', "The character's sea experience affected his etiquette awareness", 'The main character is aware of his different walk', 'Martin tries to mask his anxiety with humor' ]
}

{'running_summary': "The document is an excerpt from Jack London's 'Martin Eden' (1909), describing a character who used to work at sea, named Arthur, arriving home with a young seaside worker. Both are unsure of their etiquette in the nicest setting. The main character, Martin, has heavy arms and moves awkwardly, experiencing shame for his uncouth walk. He tries to mask his anxiety with humor, but the main character is still uncomfortable and unsure.",
 'main_ideas': ["The excerpt is from Jack London's novel 'Martin Eden'",
  'A scene shows a character arriving home with a young, seaside worker',
  'The worker is unsure of how to behave in the nicer setting',
  'The main character, who used to work at sea, is also unsure of his etiquette',
  'The main character has heavy arms and moves awkwardly',
  'The main character feels anxiety and shame due to his awareness of his different behavior'],
 'loose_ends': ["Unknown character's emotions or thoughts are not explicitly explained",
  'Id

In [24]:
summary

DocumentSummaryBase(running_summary='', main_ideas=[], loose_ends=[])

## Chunk by chapter

In [ ]:
# search all chapters titles
chapter_titles = re.findall(r"CHAPTER\s+[IVXLCDM]+|CHAPTER\s+\d+", documents[0].page_content)

In [ ]:
texts_chapters = []
start = 0

for ct in chapter_titles:
    end = documents[0].page_content.find(ct)
    if end != -1:
        texts_chapters.append(documents[0].page_content[start:end])
        start = end

In [ ]:
len(chapter_titles) == len(texts_chapters)

True

In [ ]:
import semchunk

chunk_size = 800  # adjust as needed
# chunker = semchunk.chunkerify(lambda txt: len(txt.split()), chunk_size)
chunker = semchunk.chunkerify('Qwen/Qwen3-0.6B', chunk_size)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
from langchain.docstore.document import Document

In [ ]:
documents = []
uuids = []
for chapter_num, chapter_text in enumerate(texts_chapters, start=1):
    chunks = chunker(chapter_text)
    for chunk_num, chunk_text in enumerate(chunks, start=1):
        doc = Document(
            page_content=chunk_text,
            metadata={
                "chapter_num": chapter_num,
                "chunk": chunk_num,
                "chapter_title": chapter_titles[chapter_num-1] if chapter_num-1 < len(chapter_titles) else None,
                "source": os.path.basename(src_path)
            }
        )
        doc.id = f"doc_{len(documents)}"
        uuids.append(str(uuid4()))
        documents.append(doc)

In [ ]:
documents[:5]

[Document(id='doc_0', metadata={'chapter_num': 1, 'chunk': 1, 'chapter_title': 'CHAPTER I', 'source': 'Martin_Eden_clean.txt'}, page_content='Let me live out my years in heat of blood!\nLet me lie drunken with the dreamer\'s winef Let me not see this soul-house built of mud Go toppling to the dust a vacant shrine t "\nDigitized by the Internet Archive in 2019 with funding from Kahle/Austin Foundation\nhttps://archive.org/details/martinedenOOOOunse\n" They sat idly and silently gazing with eyes that dreamed and did not see "\nBY\nAUTHOR OF " THE CALL OF THE WILD," ETC., BTC.\n\'\nCopyright, 1908, By JACK LONDON.\nSet up and electrotyped. Published September, igog. Reprinted September, October, November, 1909; January, 1910; February, 1911.\nNortooob $rrss\nJ. S. Cushing Co. - Berwick & Smith Oo. Norwood, Mass., U.S.A.\n'),
 Document(id='doc_1', metadata={'chapter_num': 2, 'chunk': 1, 'chapter_title': 'CHAPTER II', 'source': 'Martin_Eden_clean.txt'}, page_content='CHAPTER I\nThe one open

In [ ]:
vector_store.add_documents(documents, ids=uuids)

['9494c97f-4281-49dd-ad81-92608216404a',
 'cdfcef58-7d35-4dce-a755-81e46fe55f15',
 '11b6a8cb-4d8c-44ef-919e-4d0100c96598',
 'a771d079-9bb2-4497-b66f-92eec8f56866',
 '1a2d97b3-945b-426f-a4b0-c6e3b7bdbb12',
 '80df48ef-c57e-4cb6-a03f-2cb3f176c150',
 '10bb7b00-2926-4951-80a4-0ee32ce99ad2',
 'dd778b11-950c-47a6-8121-20418a3326ad',
 '7f1ed685-3e15-43e5-ac65-aa4cef3a3dfb',
 'b7d6f64f-7644-4ded-8611-5fa081065566',
 '1e6fdafe-0c67-45a4-876c-540aa53ea0ac',
 '27bcf8ee-2821-46d3-970a-3e6ac8124e70',
 'c99d9431-c4f8-4c7f-802d-2a7d87186c1e',
 '16099e98-dcdd-4acd-980b-8e9d21cd50b6',
 'eb53bb87-f4db-4b03-a62a-5c0c9e0e2481',
 'f2f8483a-637c-4f57-9d5a-c0c9a7aa6a95',
 '759eb067-f34d-4e97-97d9-954a1af141c1',
 '1371de65-8dd2-4d6c-a32a-ed01cb69512d',
 'e0f2b9bd-89da-4b66-b46d-3df89f87a9c7',
 '6f8eeef8-2e4e-46dd-a72f-948c87611410',
 'ab3e291b-6647-4184-b30a-ac2eda06e1fc',
 '5e380cff-8ce7-4a94-b0a7-c8ad249ffe7c',
 'bfda541c-3f5c-4e7b-bc16-b00c28c9d673',
 '32915fbf-68c4-4a18-b3d8-c615e5889903',
 '0d8843c7-a4ba-

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline



In [ ]:
# You can use any supported HF model, e.g., 'google/flan-t5-base', 'tiiuae/falcon-7b-instruct', etc.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0.6B",
    device_map="auto",
    token=None,  # Add your HF token here if needed
    trust_remote_code=True,
    revision="main"
)
model.generation_config.pad_token_id = tokenizer.eos_token_id


hf_pipeline = pipeline("text-generation",
                       model="Qwen/Qwen3-0.6B",
                       tokenizer=tokenizer,
                        max_new_tokens=200,
                        do_sample=True,
                        temperature=0.1,
                        top_p=0.8, # 0.9,
                        top_k=20, # 50
                        min_p=0,
                        repetition_penalty=1.3,
                        no_repeat_ngram_size=4,
                        eos_token_id=tokenizer.eos_token_id,
                        pad_token_id=tokenizer.eos_token_id)
llm = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cuda:0


In [ ]:

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

query = "Who is Ruth?"
result = qa_chain({"query": query})

print("Answer:", result['result'])
for doc in result['source_documents']:
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content)

Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

But Ruth did not care to wait ten days or two weeks. Besides, her lover was sick. The next afternoon, accompanied by Arthur, she arrived in the Morse carriage, to the unqualified delight of the Silva tribe and of all the urchins on the street, and to the consternation of Maria. She boxed the ears of the Silvas who crowded about the visitors on the tiny front porch, and in more than usual atrocious English tried to apologize for her appearance. Sleeves rolled up from soap-flecked arms and a wet gunny-sack around her waist told of the task at which she had been caught. So flustered was she by two such grand young people asking for her lodger, that she forgot to invite them to sit down in the little parlor. To enter Martin's room, they passed through the kitchen, warm and moist and steamy from the big washing in progres

In [ ]:
vector_store.similarity_search_with_relevance_scores("Who is Ruth?",
                                                                   k=3)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

def query_rag(query_text: str, vector_store=vector_store):
    results = vector_store.similarity_search_with_relevance_scores(query_text,
                                                                   k=5)


    # Check if there are any matching results or if the relevance score is too low
    if len(results) == 0:
        print(f"Unable to find matching results.")

    # Combine context from matching documents
    context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

    # Create prompt template using context and query text
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:
    {context}
    - -
    Answer the question based on the above context: {question}
    """
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    return prompt
    # Initialize OpenAI chat model
    # model = ChatOpenAI()
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B",
                                            use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen3-0.6B",
        device_map="auto",
        token=None,  # Add your HF token here if needed
        trust_remote_code=True,
        revision="main"
    )
    model.generation_config.pad_token_id = tokenizer.eos_token_id

    # if use_gpu:
    #     model = model.to("cuda")

    # Create a text generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.1,
        top_p=0.8, # 0.9,
        top_k=20, # 50
        min_p=0,
        repetition_penalty=1.3,
        no_repeat_ngram_size=4,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    # Create a LangChain wrapper around the pipeline
    llm = HuggingFacePipeline(pipeline=pipe)
    # retriever = VectorStoreRetriever(vector_store)
    qa_chain = RetrievalQA.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(
            search_kwargs={"k": 5},
        ),
        # return_source_documents=True,
    )

    # Generate response text based on the prompt
    # response_text = model.predict(prompt)
    response_text = qa_chain.invoke({"query": prompt})

    # Get sources of the matching documents
    sources = [doc.metadata.get("source", None) for doc, _score in results]

    # Format and return response including generated text and sources
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    return formatted_response, response_text

    # Let's call our function we have defined
    formatted_response, response_text = query_rag(query_text)
    # and finally, inspect our final response!
    print(response_text)

In [ ]:
results = vector_store.similarity_search_with_relevance_scores("Who is Ruth?",
                                                                   k=5)
print(results)

# Combine context from matching documents
context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])


[(Document(id='59ead614-c425-44da-96d5-bb2b75a87823', metadata={'chapter_title': 'CHAPTER I', 'source': 'C:\\Users\\NicolasCOTONI\\Work\\ReadRecall\\src\\documents\\Martin_Eden_clean.txt'}, page_content='" Ruth, this is Mr. Eden."'), 0.546766109517975), (Document(id='f2369d1e-a112-4fef-b627-2afd49dbda01', metadata={'source': 'C:\\Users\\NicolasCOTONI\\Work\\ReadRecall\\src\\documents\\Martin_Eden_clean.txt', 'chapter_title': 'CHAPTER XLV'}, page_content='She was stung by his words into realization of the puerility of her act, and yet she felt that he had magnified it unduly and was consequently resentful. They sat in silence for a long time, she thinking desperately and he pondering upon his love which had departed. He knew, now, that he had not really loved her. It was an idealized Ruth he had loved, an ethereal creature of his own creating, the bright and luminous spirit of his lovepoems. The real bourgeois Ruth, with all the bourgeois failings and with the hopeless cramp of the bour

In [ ]:
print("\n\n - -\n\n".join([doc.page_content for doc, _score in results]))

" Ruth, this is Mr. Eden."

 - -

She was stung by his words into realization of the puerility of her act, and yet she felt that he had magnified it unduly and was consequently resentful. They sat in silence for a long time, she thinking desperately and he pondering upon his love which had departed. He knew, now, that he had not really loved her. It was an idealized Ruth he had loved, an ethereal creature of his own creating, the bright and luminous spirit of his lovepoems. The real bourgeois Ruth, with all the bourgeois failings and with the hopeless cramp of the bourgeois psychology in her mind, he had never loved.
She suddenly began to speak.

 - -

" Then you did like the other women ? "
He shook his head.
" That social-settlement woman is no more than a sociological poll-parrot. I swear, if you winnowed her out between the stars, like Tomlinson, there would be found in her not one original thought. As for the portraitpainter, she was a positive bore. She'd make a good wife for the

In [ ]:
print(query_rag("Who is Ruth?"))

Human: 
    Answer the question based only on the following context:
    " Ruth, this is Mr. Eden."

 - -

" Ruth, this is Mr. Eden."

 - -

She was stung by his words into realization of the puerility of her act, and yet she felt that he had magnified it unduly and was consequently resentful. They sat in silence for a long time, she thinking desperately and he pondering upon his love which had departed. He knew, now, that he had not really loved her. It was an idealized Ruth he had loved, an ethereal creature of his own creating, the bright and luminous spirit of his lovepoems. The real bourgeois Ruth, with all the bourgeois failings and with the hopeless cramp of the bourgeois psychology in her mind, he had never loved.
She suddenly began to speak.

 - -

She was stung by his words into realization of the puerility of her act, and yet she felt that he had magnified it unduly and was consequently resentful. They sat in silence for a long time, she thinking desperately and he pondering

In [ ]:
formatted_response, response_text = query_rag("Who is Ruth?")
# and finally, inspect our final response!
print(response_text)

ValueError: too many values to unpack (expected 2)